In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from os import listdir
from os.path import isfile, join
import numpy as np 
import tensorflow as tf
from PIL import Image as img
from tensorflow.python import keras
from keras import Sequential, datasets, layers, models
import pickle as pk

In [3]:
pixels = 150
dimensions = (pixels, pixels)
bins = 5
mid_layer = 512
num_epochs = 3
learning_rate = 0.001
reduction_factor = 0.7

In [2]:
class LearningRateReducerCb(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        old_lr = self.model.optimizer.lr.read_value()
        new_lr = old_lr * reduction_factor
        print("\nEpoch: {}. Reducing Learning Rate from {} to {}".format(epoch, old_lr, new_lr))
        self.model.optimizer.lr.assign(new_lr)

In [4]:
train_images = []
train_labels = np.array([])

folder_path = '/mnt/c/NN/Diabetic Retinopathy/'

for i in range(5):
    list_dir = listdir(folder_path + str(i))
    list_dir = [f for f in list_dir]
    list_dir = sorted(list_dir)
    for j in range(0, 708, 2):
        im = img.open(folder_path + str(i) + '/' + list_dir[j])
        im = im.resize(dimensions)
        im = im.convert('L')
        im_array = np.array(im)
        im.close()
        im = img.open(folder_path + str(i) + '/' + list_dir[j + 1])
        im = im.resize(dimensions)
        im = im.convert('L')
        im_array = np.concatenate((im_array, np.asarray(im)), axis=1)
        im.close()
        train_images.append(im_array)
        train_labels = np.append(train_labels, i)

In [5]:
print(np.shape(train_images))
print(np.shape(train_labels))

(1770, 150, 300)
(1770,)


In [6]:
train_images = np.array(train_images)
train_images = np.reshape(train_images, (354, pixels, 2 * pixels, 5))

In [7]:
print(np.shape(train_images))

(354, 150, 300, 5)


In [11]:
randomize = np.arange(len(train_images))
np.random.shuffle(randomize)
new_train_images = train_images[randomize]
new_train_labels = train_labels[randomize]

new_test_images = train_images[int(len(train_labels) * 0.8 / 5):, :, :, :]
new_test_labels = train_labels[int(len(train_labels) * 0.8 / 5):]

new_train_images = train_images[:int(len(train_labels) * 0.8 / 5), :, :, :]
new_train_labels = train_labels[:int(len(train_labels) * 0.8 / 5)]

new_train_images = np.reshape(new_train_images, (len(new_train_images) * 5, pixels, 2 * pixels))
new_test_images = np.reshape(new_test_images, (len(new_test_images) * 5, pixels, 2 * pixels))

In [14]:
print(int(len(train_labels) * 0.8 / 5))
print(np.shape(new_test_images))
print(np.shape(new_test_labels))

283
(0, 150, 300, 5)
(354,)
